# Notebook 4.1: Simple Agent with Tool Calling

This notebook will introduce how to build a simple agent using Llama Stack's agent framework, enhanced with a single tool: the builtin web search tool. This capability will  allow the agent to retrieve up to date external information beyond the limits of its training data. This is an important step toward developing a more capable and autonomous agent. Make sure you have setup your environment OK in [Level 1](Level1_getting_started_with_Llama_Stack.ipynb)

## Overview

This tutorial will walk you through how to build your own AI agent who can search the web:

1. Configure a Llama Stack agent.
2. Enhance the agent by providing it access to a specific tool
2. Interact with the agent and tests its use of the web search tool.

## Prerequisites

Before starting this notebook, ensure that you have:
- Followed the instructions in the [Setup Guide](./3_Llama_Stack.ipynb) notebook. 

## 1. Setting Up this Notebook
We will start with a few imports.

In [1]:
from llama_stack_client import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from termcolor import cprint

from src.client_tools import get_pod_log
from src import lab
from src.lab import get_llama_stack_client

loaded environment variables


## 2. Using a built-in tool

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](./Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [3]:
instructions = """
    You are a helpful assistant. 
    When a user asks a question, you MUST use the websearch tool.
""" 

client = get_llama_stack_client()
agent = Agent(
    client, 
    model=lab.model_id,
    instructions=instructions,
    tools=['builtin::websearch'],
    sampling_params=lab.sampling_params
)


user_prompts = ["What is the latest OpenShift version?"]
session_id = agent.create_session("websearch-test-session")
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream='True'
    )

    for log in EventLogger().log(response):
        log.print()

Instantianted Llama Stack client

Processing user query: What is the latest OpenShift version?
inference> <tool_call>
tool_execution> Tool:brave_search Args:{'query': 'latest OpenShift version'}
tool_execution> Tool:brave_search Response:{"query": "latest OpenShift version", "top_k": [{"url": "https://learn.microsoft.com/en-us/azure/openshift/azure-redhat-openshift-release-notes", "title": "What's new with Azure Red Hat OpenShift? - Learn Microsoft", "content": "Version 4.17 - June 2025 \u00b7 OpenShift version 4.17 is now available as an Azure Red Hat OpenShift install option. For more information, see OpenShift Container", "score": 0.8051581, "raw_content": null}, {"url": "https://www.redhat.com/en/whats-new-red-hat-openshift", "title": "What's new in Red Hat OpenShift", "content": "What's New in OpenShift 4.19 2 Physical Virtual Private cloud Public cloud Edge Linux host operating system Kubernetes Run Containers and Virtual Machines", "score": 0.6824261, "raw_content": null}, {"url

## 3. Using a custom tool function

In [4]:
from pprint import pprint

from kubernetes.client.api_client import ApiClient
from kubernetes.client.rest import ApiException
from kubernetes.client import CoreV1Api
from kubernetes.config import load_incluster_config
from llama_stack_client.lib.agents.client_tool import client_tool


load_incluster_config()
api_instance = CoreV1Api(ApiClient())


def get_pod_log_test(pod_name: str, namespace: str, container_name: str):
    """
    Provide the location upon request.

    :param pod_name: The name of the target pod
    :param namespace: The name of the target namespace
    :param container_name: The name of the target container within the target pod
    :returns: Logs of the target pod
    """
    try:
        api_response = api_instance.read_namespaced_pod_log(
            pod_name, namespace, container=container_name, tail_lines=100
        )
        pprint(api_response)
    except ApiException as e:
        print("Exception when calling CoreV1Api->read_namespaced_pod_log: %s\n" % e)

In [5]:
pod_name = 'java-app-build-iyyx5z-build-pod'
namespace = 'demo-application'
container_name = 'step-s2i-build'

get_pod_log_test(pod_name, namespace, container_name)

('    at org.apache.maven.lifecycle.internal.MojoExecutor.doExecute '
 '(MojoExecutor.java:306)\n'
 '    at org.apache.maven.lifecycle.internal.MojoExecutor.execute '
 '(MojoExecutor.java:211)\n'
 '    at org.apache.maven.lifecycle.internal.MojoExecutor.execute '
 '(MojoExecutor.java:165)\n'
 '    at org.apache.maven.lifecycle.internal.MojoExecutor.execute '
 '(MojoExecutor.java:157)\n'
 '    at '
 'org.apache.maven.lifecycle.internal.LifecycleModuleBuilder.buildProject '
 '(LifecycleModuleBuilder.java:121)\n'
 '    at '
 'org.apache.maven.lifecycle.internal.LifecycleModuleBuilder.buildProject '
 '(LifecycleModuleBuilder.java:81)\n'
 '    at '
 'org.apache.maven.lifecycle.internal.builder.singlethreaded.SingleThreadedBuilder.build '
 '(SingleThreadedBuilder.java:56)\n'
 '    at org.apache.maven.lifecycle.internal.LifecycleStarter.execute '
 '(LifecycleStarter.java:127)\n'
 '    at org.apache.maven.DefaultMaven.doExecute (DefaultMaven.java:294)\n'
 '    at org.apache.maven.DefaultMaven.

## 3. Configure an agent for tool use.

- **Agent Initialization**: First we create an `Agent` instance with the desired LLM model, agent instructions and tools.

- **Instructions**: The `instructions` parameter, also referred to as the system prompt, specifies the agent's role and behavior. In this example, the agent is configured as a helpful web search assistant. It is instructed to use a tool whenever a web search is required and to respond in a friendly and helpful tone.

- **Tools**: The `tools` parameter defines the tools available to the agent. In this case, the `get_pod_log` tool is used, which enables the agent to look up logs from a pod.

- **How It Works**: When a user query is provided, the agent processes the input and determines whether a tool is required to fulfill the request. If the query involves retrieving logs from a pod, the agent invokes the `get_pod_log` tool. The tool interacts with the Kubernetes API server to fetch the container logs. This workflow ensures that the agent can handle a wide range of queries effectively.

In [7]:
instructions = """
    You are a helpful assistant. 
    When a user asks for pod logs, you MUST use the get_pod_log tool.
""" 

client = get_llama_stack_client()
agent = Agent(
    client, 
    model=lab.model_id,
    instructions=instructions,
    tools=[get_pod_log],
    sampling_params=lab.sampling_params
)


user_prompts = [
    f"What did the {container_name} container within the {pod_name} pod in namespace {namespace} log?"
]
session_id = agent.create_session("tool-test-session")
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream='True'
    )

    for log in EventLogger().log(response):
        log.print()

Instantianted Llama Stack client

Processing user query: What did the step-s2i-build container within the java-app-build-iyyx5z-build-pod pod in namespace demo-application log?
inference> <tool_call>
tool_execution> Tool:get_pod_log Args:{'pod_name': 'java-app-build-iyyx5z-build-pod', 'namespace': 'demo-application', 'container_name': 'step-s2i-build'}
tool_execution> Tool:get_pod_log Response:"    at org.apache.maven.lifecycle.internal.MojoExecutor.doExecute (MojoExecutor.java:306)\n    at org.apache.maven.lifecycle.internal.MojoExecutor.execute (MojoExecutor.java:211)\n    at org.apache.maven.lifecycle.internal.MojoExecutor.execute (MojoExecutor.java:165)\n    at org.apache.maven.lifecycle.internal.MojoExecutor.execute (MojoExecutor.java:157)\n    at org.apache.maven.lifecycle.internal.LifecycleModuleBuilder.buildProject (LifecycleModuleBuilder.java:121)\n    at org.apache.maven.lifecycle.internal.LifecycleModuleBuilder.buildProject (LifecycleModuleBuilder.java:81)\n    at org.apache

## Key Takeaways

- We've demonstrated how to set up Llama Stack agents and extended them with builtin tools (like web search) that come prepackaged with Llama Stack.
- We've shown that this simple approach can provide significantly increased functionality of existing open source LLM's. 
- This will serves as a foundational example for the more advanced examples to come involving Agentic RAG, External Tools, and complex agentic patterns.

Continue to the [next notebook](./4.2_prompt_chaining.ipynb) to learn how we can upgrade our agents to solve even more complex and multi-step tasks using advanced agentic patterns. 
